In [14]:
import numpy as np
import pandas as pd

from sklearn.model_selection import train_test_split

import torch
from torch import nn
import torch.nn.functional as F

from torch.utils.data import Dataset, DataLoader

import matplotlib.pyplot as plt

import random

In [4]:
def simulate_bivariate_abpnl(n: int) -> dict: 
    def f1(x: np.array) -> np.array:
        return x**(-1) + 10*x
    def f2(z: np.array) -> np.array:
        return z**3
    
    x = np.random.uniform(0.1, 1.1, n)
    noise = np.random.uniform(0, 5, n)
    
    z = f1(x) + noise
    y = f2(z)
    df = pd.DataFrame({'x1': x, 'x2': y})
    sim_data = {'df': df, 'noise': noise}
    
    return sim_data

In [5]:
def centering(M):
    n = M.shape[0]
    mat_ones = torch.ones((n, n))
    idendity = torch.eye(n)
    H = idendity - mat_ones/n
    
    C = torch.matmul(M, H)
    return C
    
    
def gaussian_grammat(x, sigma2=None):
    xxT = torch.squeeze(torch.matmul(x, x.T))
    x2 = torch.diag(xxT)
    xnorm = x2 - xxT + (x2 - xxT).T
    
    if sigma2 is None:
        sigma2 = torch.median(xnorm[xnorm != 0])
        
    if sigma2 == 0:
        sigma2 += 1e-16
        
    Kx = torch.exp(-xnorm/sigma2)
    
    return Kx
    
def HSIC(x, y):
    gram_x = gaussian_grammat(x)
    gram_y = gaussian_grammat(y)
    
    c = x.shape[0]**2
    hsic = torch.trace(torch.matmul(centering(gram_x), centering(gram_y)))/c
    
    return hsic

In [6]:
x = torch.Tensor(np.random.normal(size=1000).reshape((-1, 1, 1)))
y = torch.Tensor(np.random.normal(size=1000).reshape((-1, 1, 1)))
print(HSIC(x, y))
print(HSIC(x, x))

tensor(0.0002)
tensor(0.1009)


In [25]:
class MyDataset(Dataset):
    def __init__(self, data):
        self.data = data
        
    def __len__(self):
        return len(self.data)
    
    def __getitem__(self, idx):
        if torch.is_tensor(idx):
            idx = idx.tolist()
        
        data = self.data[idx, :]
        
        return data[:-1].reshape((-1, 1)), data[-1].reshape((-1, 1))
    

class Network(nn.Module):
    def __init__(self, input_dim):
        super().__init__()
        self.network = nn.Sequential(
            nn.Linear(input_dim, 5),
            nn.LeakyReLU(),
            nn.Linear(5, 5),
            nn.LeakyReLU(),
            nn.Linear(5, 5),
            nn.LeakyReLU(),
            nn.Linear(5, 5),
            nn.LeakyReLU(),
            nn.Linear(5, input_dim)
#             nn.LeakyReLU()
        )
        
        self.encode = nn.Sequential(
            nn.Linear(input_dim, 5),
            nn.LeakyReLU(),
            nn.Linear(5, 5),
            nn.LeakyReLU(),
            nn.Linear(5, 5),
            nn.LeakyReLU(),
            nn.Linear(5, 5),
            nn.LeakyReLU(),
            nn.Linear(5, input_dim)
#             nn.LeakyReLU()
        )
        self.decode = nn.Sequential(
            nn.Linear(input_dim, 5),
            nn.LeakyReLU(),
            nn.Linear(5, 5),
            nn.LeakyReLU(),
            nn.Linear(5, 5),
            nn.LeakyReLU(),
            nn.Linear(5, 5),
            nn.LeakyReLU(),
            nn.Linear(5, input_dim)
#             nn.LeakyReLU()
        )
        
    def forward(self, x, y):
        g1_x = self.network(x)
        g3_y = self.encode(y)
        y_approx = self.decode(g3_y)
        
        assert y.shape == y_approx.shape
        
        return [g1_x, y_approx, g3_y]
    
def train_model(train_loader, test_loader, num_epochs, input_dim, log_every_batch = 10):
    device = 0 if torch.cuda.is_available() else 'cpu'
    model = Network(input_dim).to(device)

    optimizer = torch.optim.Adam(model.parameters(), lr=0.01, betas=(0.9, 0.999))

    train_loss_avgs = []
    test_loss_avgs = []
    
    min_loss = 10000

    for epoch in range(num_epochs):
        model.train()
        train_loss_trace = []

        for batch, (x, y) in enumerate(train_loader):
            x = x.to(device)
            x = x.float()
            y = y.to(device)
            y = y.float()

            g1_x, y_approx, g3_y = model.forward(x, y)
            noise = g3_y - g1_x

            loss = lamb*F.mse_loss(y_approx, y) + (1-lamb)*HSIC(x, noise)

            optimizer.zero_grad()
            loss.backward()
            optimizer.step()

            train_loss_trace.append(loss.detach().item())
            if batch % log_every_batch == 0:
                print(f'Training: epoch {epoch} batch {batch} loss {loss}')

        model.eval()
        test_loss_trace = []
        for batch, (x, y) in enumerate(test_loader):
            x = x.to(device)
            x = x.float()
            y = y.to(device)
            y = y.float()

            g1_x, y_approx, g3_y = model.forward(x, y)
            noise = g3_y - g1_x

            loss = lamb*F.mse_loss(y_approx, y) + (1-lamb)*HSIC(x, noise)

            test_loss_trace.append(loss.detach().item())
            if batch % log_every_batch == 0:
                print(f'Test: epoch {epoch} batch {batch} loss {loss}')

        train_avg = np.mean(train_loss_trace)
        test_avg = np.mean(test_loss_trace)
        
        if test_avg < min_loss:
            min_loss = test_avg

        train_loss_avgs.append(train_avg)
        test_loss_avgs.append(test_avg)
        print(f'epoch {epoch} finished - avarage train loss {train_avg} ',
             f'avarage test loss {test_avg}')
        
    return train_loss_avgs, test_loss_avgs, min_loss

In [26]:
n = 1000

batch_size = 32
lamb = 0.5
num_epochs = 200

num_trials = 9

In [27]:
data = simulate_bivariate_abpnl(n)
df = data['df']
df = (df-df.mean())/df.std()
noise = data['noise']

In [28]:
def get_final_median_loss(df, num_trials):
    rand_seed = np.random.randint(0, 1000000)
    random.seed(rand_seed)
    np.random.seed(rand_seed)
    torch.manual_seed(rand_seed)

    input_dim = df.shape[1] - 1

    train, test = train_test_split(df, test_size=0.1, random_state=10, shuffle=True)

    train = np.array(train)
    test = np.array(test)

    train = MyDataset(train)
    test = MyDataset(test)
    
    train_loader = DataLoader(train, batch_size=batch_size, shuffle=True, 
                              num_workers=0, pin_memory=True)
    test_loader = DataLoader(test, batch_size=batch_size, shuffle=False,
                             num_workers=0, pin_memory=True)
    
    losses = []
    for trial in range(num_trials):
        train_loss_avgs, test_loss_avgs, min_loss = train_model(train_loader, test_loader, num_epochs, input_dim)
        losses.append(min_loss)
    
    median_loss = np.median(losses)
    return median_loss, losses

In [29]:
# for b, (x, y) in enumerate(train_loader):
#     print(b)
#     print(x.shape)
#     print(y.shape)

In [38]:
median_loss, losses = get_final_median_loss(df, num_trials)

In [31]:
median_loss

0.011459972709417343

In [32]:
losses

[0.011936204391531646,
 0.011919335345737636,
 0.011285196291282773,
 0.012202803627587855,
 0.010931709315627813,
 0.011369480635039508,
 0.011665846221148968,
 0.01126380858477205,
 0.011459972709417343]

In [33]:
df[['x2', 'x1']].head()

,x2,x1
0,-0.199210,-1.242773
1,0.099672,1.030061
2,-1.093608,-0.196601
3,1.788205,1.525318
4,0.431947,0.611544


In [37]:
median_loss_back, losses_back = get_final_median_loss(df[['x2', 'x1']], num_trials)

In [35]:
median_loss_back

0.012396272097248584

In [36]:
losses_back

[0.01262878708075732,
 0.012263277196325362,
 0.012966789654456079,
 0.012680116342380643,
 0.012383790162857622,
 0.012234172376338392,
 0.012616099382285029,
 0.012191358720883727,
 0.012396272097248584]

In [ ]:
input_dim = df.shape[1] - 1

train, test = train_test_split(df, test_size=0.1, random_state=10, shuffle=True)

train = np.array(train)
test = np.array(test)

train = MyDataset(train)
test = MyDataset(test)

train_loader = DataLoader(train, batch_size=batch_size, shuffle=True, 
                          num_workers=0, pin_memory=True)
test_loader = DataLoader(test, batch_size=batch_size, shuffle=False,
                         num_workers=0, pin_memory=True)

train_loss_avgs, test_loss_avgs, min_loss = train_model(num_epochs)

In [ ]:
fig, ax = plt.subplots(1, 1, figsize=(12,8))
ax.plot(train_loss_avgs, label='train')
ax.plot(test_loss_avgs, label='test')
ax.legend()

In [ ]:
fig, ax = plt.subplots(1, 1, figsize=(12,8))
ax.plot(train_loss_avgs[100:], label='train')
ax.plot(test_loss_avgs[100:], label='test')
ax.legend()